# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
#Load exported csv file from weatherpy
#delete the unnamed column duplicate the count id



city_df = pd.read_csv('../starter_code/output_data_file')
city_df.dropna(inplace = True) 
del(city_df["Unnamed: 0"])
city_df.head()

,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Avera,US,1651108311,33.1940,-82.5271,63.91,41,1,6.82
1,Paraíso,CR,1651108312,9.8383,-83.8656,65.41,99,100,2.82
2,Kapaa,US,1651108027,22.0752,-159.3190,82.38,87,75,12.66
3,Torbay,CA,1651108312,47.6666,-52.7314,36.28,92,77,3.00
4,Eureka,US,1651107626,40.8021,-124.1637,59.67,73,100,8.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

In [12]:
#Use the Lat and Lng as locations and Humidity as the weight.
#get locations data and humidity from the df
locations = city_df[["Latitude","Longitude"]]

humidity =city_df["Humidity"].astype(float)


In [16]:
#Add Heatmap layer to map.
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=50,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig    

    

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
narrowed_city_df = city_df.loc[(city_df["Max Temperature"] > 70) & 
                               (city_df["Max Temperature"] < 80) & 
                               (city_df["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Saint-Pierre,RE,1651108313,-21.3393,55.4781,71.29,63,0,8.05
1,Tombouctou,ML,1651108314,20.0000,-3.0000,73.76,16,0,6.78
2,Rikitea,PF,1651108320,-23.1203,-134.9692,77.25,68,0,22.24
3,Adrar,MR,1651108321,20.5022,-10.0711,70.70,21,0,8.97
4,Caravelas,BR,1651108323,-17.7125,-39.2481,74.57,89,0,4.83


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
# set up additional columns to hold information
hotel_df = narrowed_city_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

hotel_df.head()


,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Saint-Pierre,RE,1651108313,-21.3393,55.4781,71.29,63,0,8.05,
1,Tombouctou,ML,1651108314,20.0000,-3.0000,73.76,16,0,6.78,
2,Rikitea,PF,1651108320,-23.1203,-134.9692,77.25,68,0,22.24,
3,Adrar,MR,1651108321,20.5022,-10.0711,70.70,21,0,8.97,
4,Caravelas,BR,1651108323,-17.7125,-39.2481,74.57,89,0,4.83,


In [24]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "hotels",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    #get type from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # add keyword to params dict
    params['location'] = f'{lat},{lng}'

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {row["City"]}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {type} hotels is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

hotel_df.head()

Closest <class 'type'> hotels is Saint Pierre.
Missing field/result... skipping.
Closest <class 'type'> hotels is Rikitea.
Missing field/result... skipping.
Closest <class 'type'> hotels is Caravelas.
Closest <class 'type'> hotels is New Orleans.
Closest <class 'type'> hotels is Cabo Frio.
Closest <class 'type'> hotels is Saint Joseph.
Closest <class 'type'> hotels is Jagadhri.
Closest <class 'type'> hotels is Umluj.
Closest <class 'type'> hotels is Saint-Philippe.
Closest <class 'type'> hotels is Vila Velha.
Closest <class 'type'> hotels is Arlit.
Closest <class 'type'> hotels is Bandar Lengeh.
Closest <class 'type'> hotels is Grussaí.
Closest <class 'type'> hotels is Araouane.
Closest <class 'type'> hotels is Sarasota.
Closest <class 'type'> hotels is Coahuayana de Hidalgo.
Closest <class 'type'> hotels is Governador Valadares.
Closest <class 'type'> hotels is Glenwood Springs.
Closest <class 'type'> hotels is Luxor.
Closest <class 'type'> hotels is Awbari‎.


,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Saint-Pierre,RE,1651108313,-21.3393,55.4781,71.29,63,0,8.05,Saint Pierre
1,Tombouctou,ML,1651108314,20.0000,-3.0000,73.76,16,0,6.78,
2,Rikitea,PF,1651108320,-23.1203,-134.9692,77.25,68,0,22.24,Rikitea
3,Adrar,MR,1651108321,20.5022,-10.0711,70.70,21,0,8.97,
4,Caravelas,BR,1651108323,-17.7125,-39.2481,74.57,89,0,4.83,Caravelas


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [32]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))